In [11]:
import requests

response = requests.get('http://localhost:8000/v1/health/')

print(response.json())

{'healthy': True}


In [12]:
# Create a range of compost values
compost_range = [0.1, 0.2, 0.3]

# Define the parameters
params = {
    "city": "Dubai",
    "diversion_type": "compost",
    "new_value": 0.1  # Replace with the value you want to set
}

# Make the API request
response = requests.get('http://localhost:8000/v1/city_emissions/change_diversion', params=params)

# Print the response
print(response.json())

{'green': {'1960': 0.0, '1961': 0.0, '1962': 0.0, '1963': 0.0, '1964': 0.0, '1965': 0.0, '1966': 0.0, '1967': 0.0, '1968': 0.0, '1969': 0.0, '1970': 0.0, '1971': 0.0, '1972': 0.0, '1973': 0.0, '1974': 0.0, '1975': 0.0, '1976': 0.0, '1977': 0.0, '1978': 0.0, '1979': 0.0, '1980': 0.0, '1981': 0.0, '1982': 0.0, '1983': 0.0, '1984': 0.0, '1985': 0.0, '1986': 0.0, '1987': 0.0, '1988': 0.0, '1989': 0.0, '1990': 0.0, '1991': 0.0, '1992': 0.0, '1993': 0.0, '1994': 0.0, '1995': 0.0, '1996': 0.0, '1997': 0.0, '1998': 0.0, '1999': 0.0, '2000': 0.0, '2001': 0.0, '2002': 0.0, '2003': 0.0, '2004': 0.0, '2005': 0.0, '2006': 0.0, '2007': 0.0, '2008': 0.0, '2009': 0.0, '2010': 0.0, '2011': 0.0, '2012': 0.0, '2013': 0.0, '2014': 0.0, '2015': 0.0, '2016': 0.0, '2017': 0.0, '2018': 0.0, '2019': 0.0, '2020': 0.0, '2021': 0.0, '2022': 0.0, '2023': 0.0, '2024': 0.0, '2025': 0.0, '2026': 0.0, '2027': 0.0, '2028': 0.0, '2029': 0.0, '2030': 0.0, '2031': 0.0, '2032': 0.0, '2033': 0.0, '2034': 0.0, '2035': 0.0, '

In [3]:
print(response.status_code)

500
